  # task_One
  ## Analyzing output from infoTracker
  _Date: 03/27/2017_

In [6]:
import pandas as pd

In [8]:
df = pd.read_csv('/home/savy/Documents/2017_master_internship/infoTracker/prot_est_Infos.csv')

In [13]:
df.head()                                       

,Species name,Order name,Family name,#Protein,#EST
0,Catajapyx aquilonaris,Diplura,Japygidae,NaN,0
1,Orchesella cincta,Collembola,Entomobryidae,NaN,1146
2,Zootermopsis nevadensis,Blattodea,Termopsidae,14610.0,1
3,Nasutitermes exitiosus,Blattodea,Termitidae,NaN,0
4,Blattella germanica,Blattodea,Ectobiidae,NaN,8351


In [45]:
g = df.groupby(['Order name'])

In [46]:
g['#EST'].max()

Order name
Blattodea          8351
Coleoptera       185437
Collembola         1146
Diplura               0
Diptera          847942
Ephemeroptera         0
Hemiptera        219173
Hymenoptera      174597
Lepidoptera      575578
Odonata               0
Orthoptera        51188
Phasmatodea           0
Phthiraptera       1188
Plecoptera            0
Strepsiptera          0
Thysanoptera      13843
Trichoptera           0
Name: #EST, dtype: int64

## Download fasta file from NCBI

### Test 1

In [2]:
from Bio import Entrez
Entrez.email = 'savandara.besse@gmail.com'

In [3]:
handle = Entrez.esearch(db="nucest", term='drosophila melanogaster')
record = Entrez.read(handle)['IdList']
handle.close()
record

['527482192', '392859066', '345421511', '342206684', '315905243', '315844150', '315823897', '315823706', '315820860', '315809918', '315803644', '315802413', '315798677', '311205018', '311147959', '302181891', '297251322', '297251281', '297251268', '297251209']

In [5]:
print("Parsing...")
record = SeqIO.read('test.fna', "fasta")
print(record)

Parsing...
ID: JZ478789.1
Name: JZ478789.1
Description: JZ478789.1 01RQ1.12 WSSP Duckweed cDNA library Landoltia punctata cDNA clone 01RQ1.12 5' similar to protein translocation protein, mRNA sequence
Number of features: 0
Seq('AGAGAGATCGTAGAGGGATTCGGTCAAGAGTACGTTCCCCAGAGGGTTCTACCG...GCA', SingleLetterAlphabet())
